# DESAFIO III
# Clasificación de jugadores de la NBA según posición de juego
#### Grupo 5
1. Alexandra Binder
2. Andrés Montes
4. Verónica Nieves

---
## Desafío

El objetivo final a concretarse en el Desafío 3, es lograr clasificar en qué posición juega un jugador de la NBA, utilizando los métodos .... Para ello vamos a utilizar el dataset llamado `2021-2022 NBA Player Stats`, correspondiente a las estadísticas de jugadores de la NBA de la temporada regular 2021-2022 por partido. Los datos se encuentran disponibles en:

https://www.kaggle.com/vivovinco/nba-player-stats/version/1

Las features del dataset son las siguientes:

Rk : Rank

Player : Nombre

Pos : Posición

Age : Edad

Tm : Equipo

G : número de juegos

GS : juegos comenzados

MP : minutos jugados por juego

FG : encestados por partido

FGA : intentos de encesto por partido

FG% : porcentaje de encesto por partido

3P : canastas de 3 puntos por partido

3PA : intentos de canastas de 3 puntos por partido

3P% : porcentaje de canastas de 3 puntos por partido

2P : canastas de 2 puntos por partido

2PA : intentos de canasta de 2 puntos por partido

2P% : porcentaje de canastas de 2 puntos por partido

eFG% : porcentaje efectivo de canastas

FT : tiros libres por juego

FTA : intentos de tiros libres por juego

FT% : porcentaje de tiros libres

ORB : rebotes ofensivos por partido

DRB : rebote defensivo por partido

TRB : total de rebotes por juego

AST : asistencias por partido

STL : robos por juego

BLK : bloques por juego

TOV : perdedas del balón por partido

PF : faltas personales por partido

PTS : puntos por juego


#### **Objetivos del Desafío 3**
* Efectuar una limpieza del dataset provisto.
* Elegir una estrategia para balancear las categorías a clasificar
* Utilizar el modelo Naive Bayes como benchmark
* Aplicar GridSearch o RandomSearch
* Aplicar regularización a los datos
* Evaluar diferentes modelos de clasificación
* Seleccionar las métricas adecuadas según contexto de problema

---

<a id="indice"></a> 
## **Índice**

[Resumen](#section_intro)

[Módulo 1 - Primer acercamiento](#modulo_01)


[Conclusión](#modulo_14)

---

<a id="Resumen"></a> 
## Resumen
[Ir a índice](#indice)

<p style="color:yellow">XXX</p>

---
<a id="modulo_01"></a> 
## Modulo 1 - Primer acercamiento
[Ir a índice](#indice)

In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("2021-2022 NBA Player Stats V0.csv", sep=';', encoding='latin')
df.head()


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,34,20,25.3,3.4,8.1,...,0.574,2.4,5.3,7.7,1.3,0.6,0.6,1.1,2.2,7.9
1,2,Steven Adams,C,28,MEM,44,43,25.7,2.6,4.9,...,0.566,4.3,4.9,9.3,3.0,1.0,0.6,1.6,1.7,6.7
2,3,Bam Adebayo,C,24,MIA,21,21,32.9,6.9,13.3,...,0.767,2.6,7.3,10.0,3.2,1.2,0.5,2.9,3.1,18.7
3,4,Santi Aldama,PF,21,MEM,25,0,10.5,1.4,3.8,...,0.579,0.9,1.6,2.5,0.5,0.1,0.2,0.3,0.9,3.4
4,5,LaMarcus Aldridge,C,36,BRK,32,10,23.1,5.9,10.3,...,0.862,1.5,4.1,5.7,0.9,0.4,1.1,0.8,1.7,13.8


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      590 non-null    int64  
 1   Player  590 non-null    object 
 2   Pos     590 non-null    object 
 3   Age     590 non-null    int64  
 4   Tm      590 non-null    object 
 5   G       590 non-null    int64  
 6   GS      590 non-null    int64  
 7   MP      590 non-null    float64
 8   FG      590 non-null    float64
 9   FGA     590 non-null    float64
 10  FG%     590 non-null    float64
 11  3P      590 non-null    float64
 12  3PA     590 non-null    float64
 13  3P%     590 non-null    float64
 14  2P      590 non-null    float64
 15  2PA     590 non-null    float64
 16  2P%     590 non-null    float64
 17  eFG%    590 non-null    float64
 18  FT      590 non-null    float64
 19  FTA     590 non-null    float64
 20  FT%     590 non-null    float64
 21  ORB     590 non-null    float64
 22  DR

In [10]:
[x for x in df.columns if df[x].dtype == 'object']

['Player', 'Pos', 'Tm']

In [12]:
# quitamos la columna del nombre
df.drop(columns='Player', inplace=True)

In [5]:
# resumen de posición "pos" 
df_pos = pd.DataFrame({'#': df['Pos'].value_counts().values})
df_pos.index = df['Pos'].value_counts().index
df_pos['%'] = [((x/df_pos['#'].sum())*100).round(2) for x in df_pos['#']]
df_pos

,#,%
SG,145,24.58
SF,122,20.68
PG,110,18.64
PF,109,18.47
C,104,17.63


In [6]:
# resumen de equipo "Tm"
df_tm = pd.DataFrame({'#': df['Tm'].value_counts().values})
df_tm.index = df['Tm'].value_counts().index
df_tm['%'] = [((x/df_tm['#'].sum())*100).round(2) for x in df_tm['#']]
df_tm

,#,%
WAS,25,4.24
DET,24,4.07
MEM,23,3.90
MIL,23,3.90
DAL,22,3.73
ATL,22,3.73
OKC,22,3.73
DEN,21,3.56
IND,21,3.56
POR,20,3.39
